In [1]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [2]:
#import exp_filtered
data = pd.read_csv('./Extraction/experiments_filtered.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
288,745,2024-01-19 20:52:52.8522174,2024-01-19 21:22:52.8564323,general,random27,8,1,23,4,8,2,0.25,3.0,1705697572,1705699372
289,746,2024-01-19 21:26:27.3234593,2024-01-19 21:56:27.3265222,general,random27,8,1,23,4,8,2,0.25,3.0,1705699587,1705701387
290,747,2024-01-19 22:00:01.6380175,2024-01-19 22:30:01.6457071,general,random28,20,18,23,8,4,16,0.50,1.0,1705701601,1705703401
294,751,2024-01-20 00:14:24.4097982,2024-01-20 00:44:24.4126883,general,random29,18,20,8,4,6,4,0.25,3.0,1705709664,1705711464
295,752,2024-01-20 00:47:58.7269655,2024-01-20 01:17:58.7312773,general,random29,18,20,8,4,6,4,0.25,3.0,1705711678,1705713478
296,753,2024-01-20 01:21:32.6299318,2024-01-20 01:51:32.6406703,general,random30,15,4,20,8,2,8,1.00,1.0,1705713692,1705715492
299,758,2024-01-20 04:09:25.0644731,2024-01-20 04:39:25.0711445,general,random31,8,6,18,4,4,4,0.25,0.5,1705723765,1705725565
301,760,2024-01-20 05:16:34.1638368,2024-01-20 05:46:34.1666149,general,random32,3,2,5,2,2,2,0.50,1.0,1705727794,1705729594
302,761,2024-01-20 05:50:08.2461471,2024-01-20 06:20:08.2541699,general,random32,3,2,5,2,2,2,0.50,1.0,1705729808,1705731608
303,762,2024-01-20 06:23:42.6007382,2024-01-20 06:53:42.6043583,general,random33,19,8,23,4,8,2,0.25,3.0,1705731822,1705733622


In [3]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,identifier,experiment,type
0,2024-01-20 06:31:00.348127+00:00,303,762,16.0
1,2024-01-20 06:46:45.354625+00:00,303,762,16.0
2,2024-01-20 06:46:45.355063+00:00,303,762,16.0
3,2024-01-20 06:46:57.349052+00:00,303,762,16.0
0,2024-01-20 06:23:43.925455+00:00,303,762,17.0
1,2024-01-20 06:23:43.967483+00:00,303,762,17.0
2,2024-01-20 06:23:43.999768+00:00,303,762,17.0
3,2024-01-20 06:23:43.999801+00:00,303,762,17.0
4,2024-01-20 06:23:44.000139+00:00,303,762,17.0
5,2024-01-20 06:23:44.005508+00:00,303,762,17.0


In [4]:
# Transpose the table -> type and count
df = messages.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='2s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="2s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_14403/3216738472.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:04+00:00,4226.0,579.0,0.0,0.0,0.0,347.0
1,390,2023-10-04 18:52:06+00:00,3986.0,542.0,0.0,0.0,0.0,0.0
2,390,2023-10-04 18:52:08+00:00,472.0,72.0,0.0,0.0,0.0,351.0
3,390,2023-10-04 18:52:10+00:00,4241.0,576.0,0.0,0.0,0.0,347.0
4,390,2023-10-04 18:52:12+00:00,3430.0,465.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:55:14+00:00,2911.0,415.0,0.0,0.0,0.0,285.0
96,390,2023-10-04 18:55:16+00:00,1203.0,173.0,0.0,0.0,0.0,287.0
97,390,2023-10-04 18:55:18+00:00,3273.0,459.0,0.0,0.0,0.0,0.0
98,390,2023-10-04 18:55:20+00:00,2919.0,412.0,0.0,0.0,0.0,286.0


In [5]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

eventsTS.to_csv('./events2s.csv')
eventsTS.head(500)

,experiment,_time,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:04+00:00,4226.0,579.0,0.0,0.0,0.0,347.0
1,390,2023-10-04 18:52:06+00:00,3986.0,542.0,0.0,0.0,0.0,0.0
2,390,2023-10-04 18:52:08+00:00,472.0,72.0,0.0,0.0,0.0,351.0
3,390,2023-10-04 18:52:10+00:00,4241.0,576.0,0.0,0.0,0.0,347.0
4,390,2023-10-04 18:52:12+00:00,3430.0,465.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
495,390,2023-10-04 19:12:10+00:00,5008.0,695.0,0.0,0.0,0.0,420.0
496,390,2023-10-04 19:12:12+00:00,1143.0,148.0,0.0,0.0,0.0,222.0
497,390,2023-10-04 19:12:14+00:00,6154.0,843.0,0.0,0.0,0.0,195.0
498,390,2023-10-04 19:12:16+00:00,6180.0,836.0,0.0,0.0,0.0,414.0


In [6]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess2_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)

    data = pd.read_csv('../../datasets/partialTables/pub_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_pub = pd.DataFrame(data)
    
    deliverMessage = pd.concat([query_mess, deliverMessage])
    publishMessage = pd.concat([publishMessage, query_pub])

    del query_pub
    del query_mess

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-01-20 06:23:42.139365+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
1,2024-01-20 06:23:42.139457+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
2,2024-01-20 06:23:42.139631+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
3,2024-01-20 06:23:42.140159+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
4,2024-01-20 06:23:42.140332+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
5,2024-01-20 06:23:42.140430+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
6,2024-01-20 06:23:42.140436+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
7,2024-01-20 06:23:42.141264+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
8,2024-01-20 06:23:42.141383+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
9,2024-01-20 06:23:42.141725+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762


In [7]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134903+00:00,753
1,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136298+00:00,2148
2,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136886+00:00,2736
3,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137090+00:00,2940
4,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137205+00:00,3055
5,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137276+00:00,3126
6,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137353+00:00,3203
7,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137377+00:00,3227
8,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134975+00:00,308
9,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.135169+00:00,502


In [8]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='2s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_14403/22691215.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:04+00:00,1880.400662
1,390,2023-10-04 18:52:06+00:00,1670.648148
2,390,2023-10-04 18:52:08+00:00,1710.708333
3,390,2023-10-04 18:52:10+00:00,1842.387417
4,390,2023-10-04 18:52:12+00:00,1816.257895
...,...,...,...
95,390,2023-10-04 18:55:14+00:00,1586.029557
96,390,2023-10-04 18:55:16+00:00,1815.505376
97,390,2023-10-04 18:55:18+00:00,1669.377193
98,390,2023-10-04 18:55:20+00:00,2197.715686


In [9]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:04+00:00,1880.400662,4226.0,579.0,0.0,0.0,0.0,347.0
1,390,2023-10-04 18:52:06+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0
2,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0
3,390,2023-10-04 18:52:10+00:00,1842.387417,4241.0,576.0,0.0,0.0,0.0,347.0
4,390,2023-10-04 18:52:12+00:00,1816.257895,3430.0,465.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:55:14+00:00,1586.029557,2911.0,415.0,0.0,0.0,0.0,285.0
96,390,2023-10-04 18:55:16+00:00,1815.505376,1203.0,173.0,0.0,0.0,0.0,287.0
97,390,2023-10-04 18:55:18+00:00,1669.377193,3273.0,459.0,0.0,0.0,0.0,0.0
98,390,2023-10-04 18:55:20+00:00,2197.715686,2919.0,412.0,0.0,0.0,0.0,286.0


In [10]:
prop.to_csv('./propagationTime2s.csv')

In [11]:
#Bandwidth

#Load rpc dataset
# data = pd.read_csv('./Extraction/rpc.csv',header=0, index_col=0)
# rpc = pd.DataFrame(data)
# rpc = rpc.rename(columns={'rpc':'count'})

# rpc.head(10)

rpc = pd.DataFrame()


for idx, exec in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/rpc_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.134925+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.135840+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.135852+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.135928+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.135938+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.135988+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.136030+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [12]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='2s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_14403/3497163316.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [13]:
rpc.head(10)

,experiment,_time,rpc
0,390,2023-10-04 18:52:04+00:00,4940
1,390,2023-10-04 18:52:06+00:00,4494
2,390,2023-10-04 18:52:08+00:00,715
3,390,2023-10-04 18:52:10+00:00,4935
4,390,2023-10-04 18:52:12+00:00,3879
5,390,2023-10-04 18:52:14+00:00,1055
6,390,2023-10-04 18:52:16+00:00,4966
7,390,2023-10-04 18:52:18+00:00,3884
8,390,2023-10-04 18:52:20+00:00,1858
9,390,2023-10-04 18:52:22+00:00,5096


In [14]:
#Total bandwidth

# data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
# message = pd.DataFrame(data)

# idx = [3.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["type", "identifier"])

message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='2s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_14403/3096645582.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:04+00:00,580
1,390,2023-10-04 18:52:06+00:00,547
2,390,2023-10-04 18:52:08+00:00,72
3,390,2023-10-04 18:52:10+00:00,578
4,390,2023-10-04 18:52:12+00:00,465
5,390,2023-10-04 18:52:14+00:00,115
6,390,2023-10-04 18:52:16+00:00,579
7,390,2023-10-04 18:52:18+00:00,463
8,390,2023-10-04 18:52:20+00:00,205
9,390,2023-10-04 18:52:22+00:00,597


In [15]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='2s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_14403/3389687503.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:04+00:00,5520
1,390,2023-10-04 18:52:06+00:00,5041
2,390,2023-10-04 18:52:08+00:00,787
3,390,2023-10-04 18:52:10+00:00,5513
4,390,2023-10-04 18:52:12+00:00,4344
...,...,...,...
95,390,2023-10-04 18:55:14+00:00,3851
96,390,2023-10-04 18:55:16+00:00,1683
97,390,2023-10-04 18:55:18+00:00,4167
98,390,2023-10-04 18:55:20+00:00,3860


In [16]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:04+00:00,5520
1,390,2023-10-04 18:52:06+00:00,5041
2,390,2023-10-04 18:52:08+00:00,787
3,390,2023-10-04 18:52:10+00:00,5513
4,390,2023-10-04 18:52:12+00:00,4344
5,390,2023-10-04 18:52:14+00:00,1170
6,390,2023-10-04 18:52:16+00:00,5545
7,390,2023-10-04 18:52:18+00:00,4347
8,390,2023-10-04 18:52:20+00:00,2063
9,390,2023-10-04 18:52:22+00:00,5693


In [18]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwdth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/2
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

NameError: name 'gossipBandowdth' is not defined

In [ ]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/2
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

In [ ]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

In [ ]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/2

totalBand.head(500)

In [ ]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

In [ ]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.to_csv('./consDiscovery2s.csv')

consDiscovery.head(10)

In [ ]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../consolidated2s.csv')
consFinal.head(10)
